In [1]:
import pandas as pd
import re
from openpyxl import load_workbook

/Users/shiyiwang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shiyiwang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# ================== 配置区域 ==================
INPUT_FILE = "VITOL_2023_all.xlsx"
OUTPUT_FILE = "ESG_Extracted-VITOL_2023_all.xlsx"

In [3]:
# 定义关键词及对应类别 (可自由修改)
KEYWORD_CONFIG = {
    "E": [
        ["scope1"],
        ["scope1","GHG"],
        ["scope2"],
        ["scope2","total"],
        ["scope3"],
        ["electricity","consumption"],
        ["ArithmeticError","consumption"],
        ["water","consumption"],
        ["energy","consumption"],
        ["total","waste"],
        ["waste","recycled"],
        ["solar"],
        ["wind"],
        ["hydro"],
        ["bioenergy"],
        ["geothermal"],
        ["nuclear"],
        ["coal"],
        ["oil"],
        ["gas"]
    ],
    "S": [
        ["genderdiversity", "women"],
        ["management", "women"],
        ["training", "hours"],
        ["male"],
        ["female"],
        ["fatal"],
        ["jobs"],
        ["training", "hours"],
        ["man"],
        ["women"],
        ["turnover"]    
        ],
    "G": [
        ["anti","corruption"],
        ["strategy"],
        ["beneﬁtsinsurance"],
        ["board"],
        ["benefit"],
        ["board","composition"],
        ["audit"],
        ["riskassessment"],
        ["compliance"],
        ["ethics"],
        ["corruption"],
        ["bribery"],
        ["fraud"],
        ["whistleblower"],
        ["conflict", "interest"],
        ["transparency"],
        ["accountability"],
        ["independence"],
        ["inclusivity"],
        ["independence"],
        ["conflict", "interest"],
        ["transparency"],
        ["accountability"],
        ["inclusivity"]
    ]
}

In [4]:
# ==============================================

def is_number(value):
    """支持科学计数法、百分数、货币符号"""
    try:
        cleaned = re.sub(r'[^\d.eE-]', '', str(value))
        return bool(float(cleaned))
    except:
        return False

def clean_text(text):
    """保留关键符号的清洗逻辑"""
    return re.sub(r'[^\w\s_.%-]', ' ', str(text).lower()).strip()

def process_sheet(sheet):
    data = []
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value is None:
                continue
            
            # 清洗文本
            original = str(cell.value)
            cleaned = clean_text(original)
            
            # 遍历所有关键词配置
            for category, groups in KEYWORD_CONFIG.items():
                for keywords in groups:
                    # ===== G类特殊处理 =====
                    if category == "G":
                        # 使用按空格拆分后的匹配（支持子字符串匹配）
                        matched_keywords = [kw for kw in keywords if kw in cleaned.split()]
                        if not matched_keywords:
                            continue
                        data.append({
                            "Category": category,
                            "Keywords": " & ".join(keywords),
                            "Value": len(matched_keywords),  # 记录匹配到的关键字个数
                            "Source": f"{sheet.title}!{cell.coordinate}",
                            "Matched": " | ".join(matched_keywords)
                        })
                        # 一旦匹配到当前G类关键词组，不必再检查同一单元格的其他组
                        break
                    
                    # ===== E/S类处理 =====
                    else:
                        # 检查当前单元格的清洗文本是否包含所有关键词
                        if all(keyword in cleaned for keyword in keywords):
                            col_idx = cell.column
                            row_idx = cell.row
                            found_value = None
                            
                            # 向右扫描最多 3 列，避免因单位等干扰数据提取
                            for offset in range(1, 4):
                                right_cell = sheet.cell(row=row_idx, column=col_idx + offset)
                                if right_cell.value is None:
                                    continue
                                
                                if is_number(right_cell.value):
                                    # 只保留数字、科学计数法或负号
                                    value_str = re.sub(r'[^\d.e-]', '', str(right_cell.value))
                                    try:
                                        value = float(value_str)
                                    except:
                                        value = None
                                    if value is not None:
                                        found_value = value
                                        break
                            
                            if found_value is not None:
                                data.append({
                                    "Category": category,
                                    "Keywords": " & ".join(keywords),
                                    "Value": found_value,
                                    "Source": f"{sheet.title}!{cell.coordinate}"
                                })
                            # 一旦匹配到当前关键词组，跳出当前组循环
                            break
    return data


In [5]:
# 主程序
all_data = []
wb = load_workbook(INPUT_FILE)

for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    sheet_data = process_sheet(sheet)
    print(f"处理 [{sheet_name}] → 提取到 {len(sheet_data)} 条数据")
    all_data.extend(sheet_data)


处理 [Social
Governanc_e
Appendices_2] → 提取到 0 条数据
处理 [ESGperformancemetrics_9] → 提取到 4 条数据


In [6]:
# 构建DataFrame并去重
if all_data:
    df = pd.DataFrame(all_data)
    # 去重策略：G类保留最高分，E/S类保留首次出现
    df = df.sort_values(["Category", "Value"], ascending=[True, False])
    df = df.drop_duplicates(
        subset=["Category", "Keywords"], 
        keep="first"
    )
    
    # 分类输出
    with pd.ExcelWriter(OUTPUT_FILE) as writer:
        for category in ["E", "S", "G"]:
            df_cat = df[df["Category"] == category]
            df_cat.to_excel(writer, sheet_name=category, index=False)
    
    print(f"处理完成！G类数据示例：")
    print(df[df["Category"] == "G"].head())
else:
    print("警告：未提取到任何数据！")

处理完成！G类数据示例：
Empty DataFrame
Columns: [Category, Keywords, Value, Source]
Index: []


/var/folders/rz/jy82p4_54pld94ms64nt5whh0000gn/T/ipykernel_35254/2755826252.py:12: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  with pd.ExcelWriter(OUTPUT_FILE) as writer:


In [7]:
# 分类存储
datasets = {
    cat: df[df['Category'] == cat][["Keywords", "Value", "Source"]]
    for cat in ['E', 'S', 'G']
}

In [8]:
# 输出到Excel
with pd.ExcelWriter(OUTPUT_FILE) as writer:
    for category in ['E', 'S', 'G']:
        if category in datasets and not datasets[category].empty:
            datasets[category].to_excel(writer, sheet_name=category, index=False)
        else:
            # 创建空工作表占位
            pd.DataFrame({'提示': [f'未找到{category}类数据']}).to_excel(
                writer, 
                sheet_name=category, 
                index=False
            )

print("\n" + "="*50)
print(f"处理完成！共提取 {len(df)} 条有效数据")
print(f"结果文件路径: {OUTPUT_FILE}")
print("="*50)


处理完成！共提取 3 条有效数据
结果文件路径: ESG_Extracted-VITOL_2023_all.xlsx


/var/folders/rz/jy82p4_54pld94ms64nt5whh0000gn/T/ipykernel_35254/1673592645.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  with pd.ExcelWriter(OUTPUT_FILE) as writer:
